## Imports


In order for us to read the files, we need to import certain modules. Pandas and altair allow us to plot the data in the picologger file

In [ ]:
import os
import sys
import pandas as pd
import altair as alt
from datetime import datetime,date
import numpy as np

In [ ]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "itkdb"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

## User Settings

In [ ]:
input_folder = thisDir+"/../picolog_folder/"
output_file = thisDir+"/../output_file.csv"
timestamp_str = "2023-08-08 14:30:00"  # Replace this with your desired timestamp (keep format!)

## Raw data conversion code
The following data takes in all of the files from the picologger folder,performs necessary calculations on them and puts them all in one csv file.

In [ ]:
# Timestamp converter function
def TimeStampConverter(inStr, inPat):
    timeObj = None
    if inStr == "now":
        timeObj = datetime.now()
    else:
        timeObj = datetime.strptime(inStr, inPat)
    return timeObj.strftime("%Y-%m-%dT%H:%M:%SZ")

In [ ]:
### derive values from picoLogger raw data
# Perform calculations function
def perform_calculations(input_folder, output_file, timestamp_str):
    all_data = []

    timestamp = TimeStampConverter(timestamp_str, "%Y-%m-%d %H:%M:%S")

    ### loop over files
    for filename in sorted(os.listdir(input_folder)):

        # skip if not csv
        if not filename.endswith('.csv'):
            continue

        file_path = os.path.join(input_folder, filename)
        print("working on:",file_path)

        ### read in data
        data = pd.read_csv(file_path)

        ### prepare derive values
        derived ={}

        # Calculate Vin Drop and GND Drop
        derived['VIN_DROP'] = np.average(data['Vin+ Last (V)'] - data['Vin- Last (V)'])
        derived['GND_DROP'] =  np.average(data['GND+ Last (V)'] - data['GND- Last (V)'])
        
        # Calculate Resistance Vin(Ohms) and Total Resistance(mOhms)
        derived['R_VIN'] = derived['VIN_DROP'] / 10 / 5
        derived['R_GND'] = derived['GND_DROP'] / 10 / 5
        derived['EFFECTIVE_RESISTANCE'] = (derived['R_VIN'] + derived['R_GND']) * 1000
        
        # Calculate Capacitor Equivalent leakage current(nA)
        derived['HV_LEAKAGE']=  np.average(data['Capacitor leakage test Last (V)'])
        derived['LEAKAGE_CURRENT'] = ((derived['HV_LEAKAGE'] / 10) / (1 * 10 ** 6)) / 1000000
        
        # Calculate NTC value(Kohms)
        derived['NTC_VOLTAGE']=  np.average(data['NTC Last (V)'])
        derived['NTC_VALUE'] = 0.2 * 51 / derived['NTC_VOLTAGE']
                    
        # Prepare the dictionary with desired columns and values
        file_data = {
            'component': filename.replace('.csv',''),
            'componentType': 'PCB',
            'stage': 'PCB_QC',
            'testType': 'HV_LV_TEST',
            'institution': 'GL',
            'runNumber': '',
            'date': timestamp,
            'passed': 'true',
            'problems': 'false',
            'property1_value': 'B.masic',
            'property1_key': 'OPERATOR',
            'property2_value': 'INSTRUMENT',
            'property2_key': '',
            'property3_value': 'ANALYSIS_VERSION',
            'property3_key': 'picoScript: https://github.com/kwraight/picoLogger',
            'result1_key': 'VIN_DROP',
            'result1_value': derived['VIN_DROP'],
            'result2_key': 'GND_DROP',
            'result2_value': derived['GND_DROP'],
            'result3_key': 'EFFECTIVE_RESISTANCE',
            'result3_value': derived['EFFECTIVE_RESISTANCE'],
            'result4_key': 'HV_LEAKAGE',
            'result4_value': derived['HV_LEAKAGE'],
            'result5_key': 'LEAKAGE_CURRENT',
            'result5_value': derived['LEAKAGE_CURRENT'],
            'result6_key': 'NTC_VOLTAGE',
            'result6_value': derived['NTC_VOLTAGE'],
            'result7_key': 'NTC_VALUE',
            'result7_value': derived['NTC_VALUE']
        }
        ### append data
        all_data.append(file_data)
    
    ### convert all data to dataframe
    df_all=pd.DataFrame(all_data)
    # view for sanity
    display(df_all)

    ### save data if not empty
    if not df_all.empty:
        df_all.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")
    else:
        print(f"No data found :(")
        return None


## Run Conversion

In [ ]:
### run conversion
perform_calculations(input_folder, output_file, timestamp_str)
